## run this cell before starting spark cluster connection (corrects module error)

In [2]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [6]:
df_april = spark.read.json("/user/sprenner/archive/training_all_april.json")

In [7]:
df_mai = spark.read.json("/user/sprenner/archive/training_all_mai.json")

### Create temporary table view

In [8]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="hits", outputCol="bucketedFeatures")
bucketedDataApril = bucketizer.transform(df_april)
bucketedDataApril.createOrReplaceTempView("bucketApril")

bucketedDataMai = bucketizer.transform(df_mai)
bucketedDataMai.createOrReplaceTempView("bucketMai")



In [9]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucketApril \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0   1690448
1               1.0   1540848

In [10]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucketMai \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0   2271491
1               1.0   3045033

In [11]:
final_data_april = bucketedDataApril.withColumn("class_bucket", when(bucketedDataApril["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))
final_data_mai = bucketedDataMai.withColumn("class_bucket", when(bucketedDataMai["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))

In [12]:
final_data_april.createOrReplaceTempView("final_april")
final_data_mai.createOrReplaceTempView("final_mai")

In [13]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final_april \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache   1540848
1      nocache   1690448

In [14]:
spark.sql("SELECT class_bucket, count(*) \
            FROM final_mai \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache   3045033
1      nocache   2271491

In [15]:
final_data_april.describe().toPandas()

summary  account           accountHash         clientState  \
0   count  3231296               3231296             3231296   
1    mean     None  -6.862163155897912E7                None   
2  stddev     None  1.2148734164282193E9                None   
3     min      aad           -2144719344  CannotAuthenticate   
4     max   zzheng            2143592175         SystemError   

        clientStateHash                                            dataset  \
0               3231296                                            3231296   
1  -9.350691361360884E8                                               None   
2  2.5054777314131987E8                                               None   
3           -1922357674                           BBS_M1000_nominal_100418   
4            1269532611  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...   

             datasetHash datasetScope      datasetScopeHash eventType  \
0                3231296      3231296               3231296   3231296   
1  -4.5230852327388145E7         None  -4.507730727803677E8      None   
2    1.245301801667085E9         None  1.0869769945929084E9      None   
3            -2147475923    cond09_mc           -2145215727  download   
4             2147475364       valid3            2131974379  download   

  eventTypeHash eventVersion      eventVersionHash  \
0       3231296      3231296               3231296   
1    5.937808E7         None  -1.963768093741567E9   
2           0.0         None   3.851356602973206E8   
3      59378080       1.13.2           -2044360925   
4      59378080          api            1261702107   

                           filename          filenameHash  \
0                           3231296               3231296   
1                              None   -1742584.3853425994   
2                              None  1.2397161559696267E9   
3  AOD.05371635._000001.pool.root.1           -2147481484   
4            wjets_v1.15_small.root            2147482617   

        filesize_double               hits                     hostname  \
0               3231296            3231296                      3231296   
1   3.482938720038774E8  5.085354916417438                         None   
2  1.0850033994592443E9  35.12657136873025                         None   
3                   5.0                  1  130-199-21-196.dhcp.bnl.gov   
4       3.0587790406E10               1348    xenia2.nevis.columbia.edu   

          hostnameHash protocol         protocolHash  \
0              3231296  3231296              3231296   
1   9.92930343842449E7     None  4.964437819779924E8   
2  1.304920047470244E9     None  9.396917290695152E8   
3          -2142788649     davs          -1607963002   
4           2143857082      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   3231296               3231296    3231296   
1                      None  3.4601683712934375E7       None   
2                      None  1.2936656207502968E9       None   
3           AGLT2_CALIBDISK           -2135244854  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid3   

              scopeHash                           traceId  \
0               3231296                           3231296   
1  -4.625174831601209E8                          Infinity   
2  1.0843476904190636E9                               NaN   
3           -2145215727  00000a90a2c841548def8b19b974a7bf   
4            2131974379  fffffa77425f474b9fa9d590bdc6e23c   

           traceIdHash                    traceIp            traceIpHash  \
0              3231296                    3231296                3231296   
1   -620846.8997167081                       None  -2.1964318650645006E8   
2  1.239984755536332E9                       None   1.2257798827174745E9   
3          -2147483615  2001:1458:201:e3::100:107            -2143850866   
4           2147482933        ::ffff:95.103.58.33             2146432649   

     traceTimeentryU

In [16]:
final_data_mai.describe().toPandas()

summary   account          accountHash     clientState  \
0   count   5316524              5316524         5316524   
1    mean      None  5.676034578304038E7            None   
2  stddev      None  1.078238184743408E9            None   
3     min  aalfonsi          -2144719344  AttributeError   
4     max    zzheng           2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2  3.3932279989536566E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

           datasetHash datasetScope      datasetScopeHash eventType  \
0              5316524      5316524               5316524   5316524   
1  8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.264352081440324E9         None   1.159227636261917E9      None   
3          -2147477854    cond09_mc           -2145215727  download   
4           2147445885       valid2            2144116684  download   

  eventTypeHash eventVersion       eventVersionHash  \
0       5316524      5316524                5316524   
1    5.937808E7         None  -2.0386347870917923E9   
2           0.0         None   1.2667515763379407E8   
3      59378080       1.13.2            -2044360925   
4      59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double                hits                  hostname  \
0               5316524             5316524                   5316524   
1  2.8167336660135007E8   6.517046664324284                      None   
2  1.0094883391407571E9  13.920048576048561                      None   
3                   2.0                   1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                 249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325294E9     None  9.234807216961869E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2   1.1593529202301002E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

            traceIdHash                    traceIp            traceIpHash  \
0               5316524                    5316524                5316524   
1   -254286.60280984343                       None  -4.1963757976758495E7   
2  1.2400714422504263E9                       None    1.201045326958785E9   
3           -2147483583  2001:1458:201:e4::100:28c            -2143850866   
4            2147482661   

In [17]:
final_data_april = final_data_april.drop("traceTimeentry")
final_data_mai = final_data_mai.drop("traceTimeentry")

In [18]:
categoricalFeatures = ['account', 'clientState', 'dataset', 'datasetScope', 'eventType', 'eventVersion', 'filename', 
                       'hostname', 'protocol', 'remoteSite', 'scope', 'traceId', 'traceIp', 'uuid']

encoded_april = final_data_april
encoded_mai = final_data_mai



for elem in categoricalFeatures:
    encoded_april = encoded_april.withColumn(elem + "Hash", hash(encoded_april[elem]))
    encoded_mai = encoded_mai.withColumn(elem + "Hash", hash(encoded_mai[elem]))

In [19]:
encoded_april.describe().toPandas()

summary  account           accountHash         clientState  \
0   count  3231296               3231296             3231296   
1    mean     None  -6.862163155897912E7                None   
2  stddev     None  1.2148734164282184E9                None   
3     min      aad           -2144719344  CannotAuthenticate   
4     max   zzheng            2143592175         SystemError   

        clientStateHash                                            dataset  \
0               3231296                                            3231296   
1  -9.350691361360884E8                                               None   
2  2.5054777314132005E8                                               None   
3           -1922357674                           BBS_M1000_nominal_100418   
4            1269532611  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...   

             datasetHash datasetScope      datasetScopeHash eventType  \
0                3231296      3231296               3231296   3231296   
1  -4.5230852327388145E7         None  -4.507730727803677E8      None   
2    1.245301801667085E9         None   1.086976994592909E9      None   
3            -2147475923    cond09_mc           -2145215727  download   
4             2147475364       valid3            2131974379  download   

  eventTypeHash eventVersion      eventVersionHash  \
0       3231296      3231296               3231296   
1    5.937808E7         None  -1.963768093741567E9   
2           0.0         None   3.851356602973206E8   
3      59378080       1.13.2           -2044360925   
4      59378080          api            1261702107   

                           filename          filenameHash  \
0                           3231296               3231296   
1                              None   -1742584.3853425994   
2                              None  1.2397161559696271E9   
3  AOD.05371635._000001.pool.root.1           -2147481484   
4            wjets_v1.15_small.root            2147482617   

       filesize_double               hits                     hostname  \
0              3231296            3231296                      3231296   
1  3.482938720038774E8  5.085354916417438                         None   
2  1.085003399459244E9  35.12657136873021                         None   
3                  5.0                  1  130-199-21-196.dhcp.bnl.gov   
4      3.0587790406E10               1348    xenia2.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               3231296  3231296              3231296   
1    9.92930343842449E7     None  4.964437819779924E8   
2  1.3049200474702442E9     None  9.396917290695155E8   
3           -2142788649     davs          -1607963002   
4            2143857082      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   3231296               3231296    3231296   
1                      None  3.4601683712934375E7       None   
2                      None  1.2936656207502983E9       None   
3           AGLT2_CALIBDISK           -2135244854  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid3   

              scopeHash                           traceId  \
0               3231296                           3231296   
1  -4.625174831601209E8                          Infinity   
2  1.0843476904190636E9                               NaN   
3           -2145215727  00000a90a2c841548def8b19b974a7bf   
4            2131974379  fffffa77425f474b9fa9d590bdc6e23c   

           traceIdHash                    traceIp            traceIpHash  \
0              3231296                    3231296                3231296   
1   -620846.8997167081                       None  -2.1964318650645006E8   
2  1.239984755536331E9                       None   1.2257798827174745E9   
3          -2147483615  2001:1458:201:e3::100:107            -2143850866   
4           2147482933        ::ffff:95.103.58.33             2146432649   

     traceTimeentryU

In [20]:
encoded_mai.describe().toPandas()

summary   account           accountHash     clientState  \
0   count   5316524               5316524         5316524   
1    mean      None   5.676034578304038E7            None   
2  stddev      None  1.0782381847434082E9            None   
3     min  aalfonsi           -2144719344  AttributeError   
4     max    zzheng            2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953656E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

            datasetHash datasetScope      datasetScopeHash eventType  \
0               5316524      5316524               5316524   5316524   
1   8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.2643520814403243E9         None   1.159227636261917E9      None   
3           -2147477854    cond09_mc           -2145215727  download   
4            2147445885       valid2            2144116684  download   

          eventTypeHash eventVersion       eventVersionHash  \
0               5316524      5316524                5316524   
1            5.937808E7         None  -2.0386347870917923E9   
2  7.106701486539487E-9         None   1.2667515763379408E8   
3              59378080       1.13.2            -2044360925   
4              59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double               hits                  hostname  \
0               5316524            5316524                   5316524   
1  2.8167336660135007E8  6.517046664324284                      None   
2   1.009488339140757E9  13.92004857604856                      None   
3                   2.0                  1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325299E9     None  9.234807216961868E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2      1.1593529202301E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

           traceIdHash                    traceIp            traceIpHash  \
0              5316524                    5316524                5316524   
1  -254286.60280984343                       None  -4.1963757976758495E7   
2  1.240071442250426E9                       None   1.2010453269587848E9   
3          -2147483583  2001:1458:201:e4::100:28c    

In [23]:
#predictions.write.format("json").save("/user/sprenner/forest_training_all_april.json")

In [24]:
final_data_april = encoded_april.orderBy("traceTimeentryUnix", ascending=True)
final_data_april.createOrReplaceTempView('final_april')
final_data_april = spark.sql("SELECT ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as idx, * FROM final_april")

## Bucketing End

In [27]:

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["accountHash","clientStateHash", "datasetScopeHash",
                                       "filesize_double", "protocolHash",
                                       "scopeHash","traceTimeentryUnix",
                                       "transferStart"], outputCol="features")

output_april = assembler.transform(final_data_april)
#output_april = output_april_all.select("bucketedFeatures", "features")
output_april = output_april.drop("accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
                                       "eventVersionHash","filenameHash","hostnameHash","protocolHash",
                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash",
                                       "uuidHash")
output_mai = assembler.transform(encoded_mai)
#output_mai = output_mai_all.select("bucketedFeatures", "features")
output_mai = output_mai.drop("accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
                                       "eventVersionHash","filenameHash","hostnameHash","protocolHash",
                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash",
                                       "uuidHash")
        
# ACHTUNG: Random Split gibt unterschiedliche Ergebnisse mit selbem Seed wenn die Spalten nicht die selben sind!!
# Random also auch von Spalten abhängig und spricht nicht nur die Zeilennummer an 
train_data, test_data = output_april.randomSplit([.8,.2],seed=1234)

#output_april.createOrReplaceTempView('out_april')
#train_data = spark.sql("SELECT * FROM out_april WHERE idx <= 2585036")
#test_data = spark.sql("SELECT * FROM out_april WHERE idx > 2585036")

train_data = train_data.drop("idx")
test_data = test_data.drop("idx")

rf = RandomForestClassifier(labelCol='bucketedFeatures', featuresCol='features',numTrees=2000, seed=1234)
model = rf.fit(train_data)
predictions_april = model.transform(test_data)
predictions_mai = model.transform(output_mai)

# possible metrics: f1|weightedPrecision|weightedRecall|accuracy
evaluator = MulticlassClassificationEvaluator(labelCol="bucketedFeatures", predictionCol="prediction", metricName="accuracy")
accuracy_april = evaluator.evaluate(predictions_april)
print("Test Error April = %g" % (1.0 - accuracy_april))

accuracy_mai = evaluator.evaluate(predictions_mai)
print("Test Error Mai = %g" % (1.0 - accuracy_mai))

print(str(model.featureImportances))

#Test Error April Random = 0.208719
#Test Error Mai Random = 0.296585

#Test Error April Ordered = 
#Test Error Mai Ordered = 

Py4JJavaError: An error occurred while calling o660.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 75.0 failed 4 times, most recent failure: Lost task 0.3 in stage 75.0 (TID 3297, p05153074925396.cern.ch, executor 88): java.lang.IllegalArgumentException: Size exceeds Integer.MAX_VALUE
	at sun.nio.ch.FileChannelImpl.map(FileChannelImpl.java:869)
	at org.apache.spark.storage.DiskStore$$anonfun$getBytes$4.apply(DiskStore.scala:125)
	at org.apache.spark.storage.DiskStore$$anonfun$getBytes$4.apply(DiskStore.scala:124)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1337)
	at org.apache.spark.storage.DiskStore.getBytes(DiskStore.scala:126)
	at org.apache.spark.storage.BlockManager.getLocalValues(BlockManager.scala:520)
	at org.apache.spark.storage.BlockManager.get(BlockManager.scala:693)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:753)
	at org.apache.spark.rdd.RDD.getOrCompute(RDD.scala:334)
	at org.a

In [ ]:
predictions_april.limit(10).toPandas()

In [ ]:
predictions_april = predictions_april.withColumn("filesize", predictions_april["filesize_double"].cast(IntegerType()))

In [ ]:
df_april_sim_eval = predictions_april.select('uuid', 'account', 'datasetScope', 'dataset', 'scope', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_april.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [ ]:
df_april_sim_eval.limit(10).toPandas()

In [ ]:
#df_april_sim_eval.write.csv('/user/sprenner/forest_april_all__eval_predictions_numTrees2000_seed1234_cache01.csv', sep='\t')